In [55]:
import pandas as pd
import numpy as np


data = pd.read_pickle('dt_trxpse_V3.pkl.gzip')

# Dado que esta fase se acerca al entrenamiento, es necesario depurar transacciones que no puedan ser usadas.

Los criterios para esto son:
* Si no tiene fecha, se elimina.
* Si no se tiene valor, se elimina.
* Si no tiene cliente, se elimina.

In [56]:
data_clean = data.dropna(subset=['fecha_usable','id_cliente', 'valor_trx'])

In [ ]:
#Muestreo aleatorio del 0.01
small_df = df.sample(frac=0.01,random_state=11).compute()
#small_df.fecha_usable=small_df.fecha_usable.astype(np.datetime64)

In [ ]:
#small_df.id_cliente.value_counts()
test_df = small_df[small_df.id_cliente.isin(['269115','187977','26729','157323','131290','254083','78033'])]

In [22]:
###PRIMERA FUNCIÓN
#Obtiene las categorías de sectores y subsectores en las que el cliente del registro más 
#gastó (hasta un máximo de 3) por cada mes.
#df: dataframe con los datos de los clientes
#column_name: columna a rankear
def get_df_categories_rank(df,column_name='sector'):
    sized_df=df.groupby(['id_cliente',pd.Grouper(key='fecha_usable',freq='M'),column_name],as_index=False).size().unstack(fill_value=0).sort_index(level=['id_cliente','fecha_usable'])
    cantidad_columnas = sized_df.shape[1]
    sized_df.insert(cantidad_columnas, column='NINGUNO',value=0, allow_duplicates=True)
    sized_df.insert(cantidad_columnas+1, column='NINGUNO',value=0, allow_duplicates=True)
    sized_df.insert(cantidad_columnas+2, column='NINGUNO',value=0, allow_duplicates=True)
    sized_df_index = sized_df.index
    sized_df_columns = sized_df.columns
    column_ranks = sized_df.apply(np.argsort, axis=1)
    
    #ranked_columns = sized_df.columns.to_series()[column_ranks.values[:,::-1][:,:3]]
    del sized_df
    ranked_columns = sized_df_columns.to_series()[column_ranks.values[:,::-1][:,:3]]
    
    del column_ranks
    return pd.DataFrame(ranked_columns, index=sized_df_index, columns=['primer_{}'.format(column_name),'segundo_{}'.format(column_name),'tercer_{}'.format(column_name)]).reset_index(level=['id_cliente','fecha_usable'])

In [3]:
#Devuelve el gasto histórico TOTAL a la fecha del registro. La frecuencia se controla con el parámetro "freq"
def get_cumulative_values(df,freq,column_name='valor_trx'):
    return df.groupby(['id_cliente',pd.Grouper(key='fecha_usable',freq=freq)],as_index=True)['valor_trx'].sum().sort_index(level=['id_cliente','fecha_usable']).groupby('id_cliente').cumsum()

#SEGUNDA FUNCIÓN
#Esta función calcula el gasto acumulado en lo corrido del mes. Lo hace hallando el gasto acumulado total al día
#de la transacción y restándole el gasto acumulado a inicio del mes.
#Este gasto acumulado INCLUYE
def gasto_mes_corrido(df):
    acumulado_dias = get_cumulative_values(df,'D')
    acumulado_meses = get_cumulative_values(df,'M').to_frame(name='valor_trx_mes')
    fin_mes_previo = (acumulado_dias.index.get_level_values('fecha_usable').to_period('M').astype('datetime64[D]')-np.timedelta64(1,unit='D')).to_period('D')
    acumulado_meses['id_cliente_key'] = acumulado_meses.index.get_level_values('id_cliente')
    acumulado_meses['fin_mes_previo'] = acumulado_meses.index.get_level_values('fecha_usable')
    df_final = pd.DataFrame({'valor_trx_acum':acumulado_dias,'id_cliente_key':acumulado_dias.index.get_level_values('id_cliente'),'fecha_usable':acumulado_dias.index.get_level_values('fecha_usable'),'fin_mes_previo':fin_mes_previo.astype('datetime64[D]')},index=acumulado_dias.index)
    df_final = pd.merge_asof(df_final.sort_values(['fin_mes_previo']),acumulado_meses.sort_values(['fin_mes_previo']),on=['fin_mes_previo'],by='id_cliente_key').sort_values(['id_cliente_key','fecha_usable'])
    df_final['valor_trx_mes'].fillna(0,inplace=True)
    df_final['trx_mes_corrido'] = df_final.valor_trx_acum-df_final.valor_trx_mes
    
    return df_final[['id_cliente_key','fecha_usable','trx_mes_corrido']]

In [30]:
#Ejemplo de uso función 1
r1 = get_df_categories_rank(data_clean,'subsector')
##Para unir la base, se debe usar pd.merge_asof
##OJO, NO PUDEN HABER NaN en ninguna en las columnas "fecha_usable" para poder hacer este merge
#ejemplo del merge_asof
#pd.merge_asof(data_clean.sort_values(['fecha_usable']),r1.sort_values(['fecha_usable']),on='fecha_usable', by='id_cliente')


In [31]:
r2 = get_df_categories_rank(data_clean,'descripcion')

In [57]:
data_clean_merged =  pd.merge_asof(data_clean.sort_values(['fecha_usable']),r1.sort_values(['fecha_usable']),on='fecha_usable', by='id_cliente')

In [58]:
data_clean_merged = pd.merge_asof(data_clean_merged.sort_values(['fecha_usable']),r2.sort_values(['fecha_usable']),on='fecha_usable', by='id_cliente')

In [21]:
#Ejemplo de uso función 2
data_gasto = gasto_mes_corrido(data_clean)
#Aquí solo basta hacer un merge corriente

In [37]:
data_gasto.columns = ['id_cliente','fecha_usable','trx_mes_corrido']

In [38]:
data_gasto.head()

,id_cliente,fecha_usable,trx_mes_corrido
0,1,2016-09-14,2155637.15
470262,1,2016-10-10,1174274.69
724648,1,2016-11-10,1765195.73
969356,1,2016-12-13,1525157.17
1011211,1,2017-01-09,2502761.42


In [59]:
data_clean_merged_final = pd.merge_asof(data_clean_merged.sort_values(['fecha_usable']),data_gasto.sort_values(['fecha_usable']),on='fecha_usable', by='id_cliente')

In [61]:
data_clean_merged_final.columns

Index(['id_trn_ach', 'id_cliente', 'valor_trx', 'ref1', 'ref2', 'ref3',
       'sector', 'subsector', 'descripcion', 'fecha_usable', 'ref1_exp_nlp',
       'ref2_exp_nlp', 'ref3_exp_nlp', 'primer_subsector', 'segundo_subsector',
       'tercer_subsector', 'primer_descripcion', 'segundo_descripcion',
       'tercer_descripcion', 'trx_mes_corrido'],
      dtype='object')

In [65]:
data_clean_merged_final[['primer_subsector', 'segundo_subsector',
       'tercer_subsector', 'primer_descripcion', 'segundo_descripcion',
       'tercer_descripcion']] = data_clean_merged_final[['primer_subsector', 'segundo_subsector',
       'tercer_subsector', 'primer_descripcion', 'segundo_descripcion',
       'tercer_descripcion']].fillna(value='NINGUNO')

In [66]:
data_clean_merged_final.head()

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion,fecha_usable,ref1_exp_nlp,ref2_exp_nlp,ref3_exp_nlp,primer_subsector,segundo_subsector,tercer_subsector,primer_descripcion,segundo_descripcion,tercer_descripcion,trx_mes_corrido
0,215120528,17734,277551.29,Pago OnLine Amway,CC,NaN,NaN,NaN,NaN,2016-09-01 00:00:04,"[pago, online, amway]",cc,cc,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,863895.90
1,215120525,248749,165555.19,Pago por iRecaudo de Claro Soluciones Móviles,NaN,NaN,NaN,NaN,NaN,2016-09-01 00:00:50,"[pago, irecaudo, soluciones, moviles]",vacio,vacio,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,691192.76
2,215120522,11856,32229.02,Pago de Saldo,NaN,NaN,MEDIOS DE COMUNICACION,TELEFONIA FIJA,Servicios de telefonía fija,2016-09-01 00:00:59,"[pago, saldo]",vacio,vacio,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,32229.02
3,215120581,212722,381295.37,CC,NaN,NaN,NaN,NaN,NaN,2016-09-01 00:01:08,cc,vacio,vacio,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,488658.23
4,215120539,283039,321851.13,Vuelo MDE PSO,CC,NaN,NaN,NaN,NaN,2016-09-01 00:01:38,"[vuelo, mde, pso]",cc,cc,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,321851.13


In [67]:
data_clean_merged_final.to_pickle('dt_trxpse_V4.pkl.gzip')

# Con esto la data queda enriquecida con el monto que gasto el mes anterior, sus transacciones preferidas.

In [ ]:
#####DE AQUÍ PARA ABAJO, TODO ES CÓDIGO FALLIDO Y CÓDIGO BORRADOR
acumulado_dias = get_cumulative_values(df,'D')
acumulado_meses = get_cumulative_values(df,'M').to_frame(name='valor_trx_mes')
fin_mes_previo = (acumulado_dias.index.get_level_values('fecha_usable').to_period('M').astype('datetime64[D]')-np.timedelta64(1,unit='D')).to_period('D')
acumulado_meses['id_cliente_key'] = acumulado_meses.index.get_level_values('id_cliente')
acumulado_meses['fin_mes_previo'] = acumulado_meses.index.get_level_values('fecha_usable')
df_final = pd.DataFrame({'valor_trx_acum':acumulado_dias,'id_cliente_key':acumulado_dias.index.get_level_values('id_cliente'),'fecha_usable':acumulado_dias.index.get_level_values('fecha_usable'),'fin_mes_previo':fin_mes_previo.astype('datetime64[D]')},index=acumulado_dias.index)
pd.merge_asof(df_final.sort_values(['fin_mes_previo']),acumulado_meses.sort_values(['fin_mes_previo']),on=['fin_mes_previo'],by='id_cliente_key').sort_values(['id_cliente_key','fecha_usable'])['valor_trx_mes'].fillna(0)


In [ ]:
#df_final.join(acumulado_meses, on=['id_cliente_key','fin_mes_previo'], rsuffix='_x')
#df_final.merge(acumulado_meses,on=['id_cliente','fin_mes_previo'],how='left',left_index=False, right_index=False)
#pd.merge_asof(df_final.sort_values(['fin_mes_previo']),acumulado_meses.sort_values(['fin_mes_previo']),on=['fin_mes_previo'],by='id_cliente_key')
#df_final['fin_mes_previo']
#acumulado_meses

In [ ]:
###### OTROS MÉTODOS QUE NO FUNCIONARON


#Obtiene las categorías de sectores y subsectores en las que el cliente del registro más 
#gastó (hasta un máximo de max_categorias) en la ventana de tiempo comprendida
#entre fecha_base y ventana_dias días antes
#row: pandas.Series con información de una fila de un dataframe
#ventana_dias_int: Cantidad de días anteriores a la fecha del registro desde donde se
#empezará a hacer la jerarquización de las categorías
#df: DataFrame en donde se hará la búsqueda de los registros para poder hacer la jerarquización.
def top_categories_single(row,ventana_dias_int,df, max_categorias=3):
    ventana_dias = np.timedelta64(ventana_dias_int,'D')
    id = row['id_cliente']
    fecha_base =row['fecha_usable']
    transacciones_cliente = df[df.id_cliente == id]
    transacciones_cliente = transacciones_cliente[(transacciones_cliente.fecha_usable>=(fecha_base-ventana_dias)) & (transacciones_cliente.fecha_usable<fecha_base)]
    top_categorias = transacciones_cliente['sector'].value_counts().index#.groupby('sector')['sector'].value_counts().index
    top_subcategorias = transacciones_cliente['subsector'].value_counts().index#.groupby('subsector')['subsector'].value_counts().index
    cant_top_categorias = len(top_categorias)
    cant_top_subcategorias = len(top_subcategorias)
    return pd.Series([top_categorias[i] if i < cant_top_categorias else np.nan for i in range(max_categorias)] +
                     [top_subcategorias[i] if i < cant_top_subcategorias else np.nan for i in range(max_categorias)])
#     if len(top_categorias)>max_categorias:
#         top_categorias = top_categorias[0:max_categorias-1]
#     if len(top_subcategorias)>max_categorias:
#         top_subcategorias = top_subcategorias[0:max_categorias-1]
    
    
#     return top_categorias.index.get_values(), top_subcategorias.index.get_values()

In [ ]:
#ejemplo de uso:
max_category_columns = test_df.apply(top_categories_single,axis=1,ventana_dias_int=30,df=small_df)
max_category_columns.columns = ['max_sector_1','max_sector_2','max_sector_3','max_subsector_1','max_subsector_2','max_subsector_3']
#después hacer un join
test_df.join(max_category_columns)

In [ ]:
#Otra solución en la que estoy trabajando. Podría ser más rápida
#small_df.groupby([]).size().unstack(fill_value=0)
#r =test_df.groupby(['id_cliente'],as_index=False).resample('D',on='fecha_usable')#['sector'].count()#['sector']#.size().unstack(fill_value=0)#.groupby(['id_cliente']).count()#.size().unstack(fill_value=0)
sized_df=small_df.groupby(['id_cliente',pd.Grouper(key='fecha_usable',freq='M'),'sector'],as_index=False).size().unstack(fill_value=0).sort_index(level=['id_cliente','fecha_usable'])#.resample('D',level='fecha_usable').sum()

In [ ]:
#sized_df['Month'] = sized_df.index.get_level_values(level=1)
cantidad_columnas = sized_df.shape[1]
#cantidad_columnas
sized_df.insert(cantidad_columnas, column='NINGUNO',value=0, allow_duplicates=True)
sized_df.insert(cantidad_columnas+1, column='NINGUNO',value=0, allow_duplicates=True)
sized_df.insert(cantidad_columnas+2, column='NINGUNO',value=0, allow_duplicates=True)

In [ ]:
sized_df.columns

In [ ]:
#[['GOBIERNO','MEDIOS DE COMUNICACION']
arank = sized_df.apply(np.argsort, axis=1)

In [ ]:
arank.head()

In [ ]:
sized_df.head()

In [ ]:
ranked_cols = sized_df.columns.to_series()[arank.values[:,::-1][:,:3]]

In [ ]:
new_frame = pd.DataFrame(ranked_cols, index=sized_df.index)

In [ ]:
new_frame

In [ ]:
sized_df[['GOBIERNO','MEDIOS DE COMUNICACION']].idxmax(1)

In [ ]:
small_df[small_df.id_cliente == '99745']

In [ ]:
#Obtiene las categorías de sectores y subsectores en las que el cliente con id_cliente id más 
#gastó (hasta un máximo de max_categorias) en la ventana de tiempo comprendida
#entre fecha_base y ventana_dias días antes.

def top_categories(df,id, fecha_base, ventana_dias, max_categorias=5):
    transacciones_cliente = df[df.id_cliente == id]
    transacciones_cliente = transacciones_cliente[(transacciones_cliente.fecha_usable>=(fecha_base-ventana_dias)) & (transacciones_cliente.fecha_usable<=fecha_base)]
    top_categorias = transacciones_cliente.groupby('sector')['sector'].count()
    top_subcategorias = transacciones_cliente.groupby('subsector')['subsector'].count()

    if len(top_subcategorias)>max_categorias:
        top_categorias = top_categorias[0:max_categorias-1]
        top_subcategorias = top_subcategorias[0:max_categorias-1]
    
    
    return top_categorias.index.get_values(), top_subcategorias.index.get_values()

#Obtiene el total gastado por el cliente con id_cliente id en lo que va del mes de fecha_base.
def total_consumo(df,id, fecha_base):
    ventana = fecha_actual-fecha_actual.astype('datetime64[M]')
    transacciones_cliente = df[df.id_cliente == id]
    transacciones_cliente = transacciones_cliente[(transacciones_cliente.fecha_usable>=(fecha_base-ventana_dias)) & (transacciones_cliente.fecha_usable<=fecha_base)]
    conusmo_mes = transacciones_cliente['valor_trx'].sum()
    #top_subcategorias = transacciones_cliente.groupby('subsector')['subsector'].count()
    return conusmo_mes

In [ ]:
#Obtiene las categorías de sectores y subsectores en las que el cliente con id_cliente id más 
#gastó (hasta un máximo de max_categorias) en la ventana de tiempo comprendida
#entre fecha_base y ventana_dias días antes. Esta función recibe un dataframe dask.
def top_categories_dask(df,id, fecha_base, ventana_dias):
    transacciones_cliente = df[df.id_cliente == id]
    transacciones_cliente = transacciones_cliente[(transacciones_cliente.fecha_usable>=(fecha_base-ventana_dias)) & (transacciones_cliente.fecha_usable<=fecha_base)].compute()
    top_categorias = transacciones_cliente.groupby('sector')['sector'].count()
    top_subcategorias = transacciones_cliente.groupby('subsector')['subsector'].count()

    if len(top_subcategorias)>max_categorias:
        top_categorias = top_categorias[0:max_categorias-1]
        top_subcategorias = top_subcategorias[0:max_categorias-1]
        
    return top_categorias.index.get_values(), top_subcategorias.index.get_values()

#Obtiene el total gastado por el cliente con id_cliente id en lo que va del mes de fecha_base.
#Esta función recibe un dataframe de dask
def total_consumo_dask(df,id, fecha_base):
    ventana = fecha_actual-fecha_actual.astype('datetime64[M]')
    #transacciones_cliente = df[df.id_cliente == id]
    #transacciones_cliente = transacciones_cliente[(transacciones_cliente.fecha_usable>=(fecha_base-ventana_dias)) & (transacciones_cliente.fecha_usable<=fecha_base)].compute()
    transacciones_cliente = df[(df.id_cliente == id)&(df.fecha_usable>=(fecha_base-ventana_dias)) & (df.fecha_usable<=fecha_base)].compute()
    conusmo_mes = transacciones_cliente['valor_trx'].sum()
    #top_subcategorias = transacciones_cliente.groupby('subsector')['subsector'].count()
    return conusmo_mes

In [ ]:
##preparación de datos para prueba
id = '26729'
fecha_actual = np.datetime64('2018-04-20')
ventana_dias = 10



In [ ]:
sector, subsector = top_categories(small_df,id,fecha_actual,10)
print(sector)
print(subsector)

In [ ]:
#scraper páginas amarillas
